In [3]:
import tensorflow as tf
tf.config.run_functions_eagerly(True)
from tensorflow.keras import layers,models,metrics,optimizers
import matplotlib.pyplot as plt
import numpy as np
import os
import math
%load_ext tensorboard

In [4]:
CNNmodel = models.Sequential()
CNNmodel.add(layers.Conv2D(128,(5,5),strides = 1, activation ="relu",padding = "valid" ,input_shape=(17,17,14)))
CNNmodel.add(layers.Conv2D(128,(3,3),strides =1,padding = "same",activation = "relu",use_bias = 1))
CNNmodel.add(layers.Conv2D(128,(3,3),strides =1,padding = "same",activation = "relu",use_bias = 1))
CNNmodel.add(layers.Conv2D(128,(3,3),strides =1,padding = "same",activation = "relu",use_bias = 1))
CNNmodel.add(layers.Conv2D(128,(3,3),strides =1,padding = "same",activation = "relu",use_bias = 1))
CNNmodel.add(layers.Conv2D(128,(3,3),strides =1,padding = "same",activation = "relu",use_bias = 1))
CNNmodel.add(layers.Conv2D(128,(3,3),strides =1,padding = "same",activation = "relu",use_bias = 1))
CNNmodel.add(layers.Conv2D(128,(3,3),strides =1,padding = "same",activation = "relu",use_bias = 1))
CNNmodel.add(layers.Conv2D(128,(3,3),strides =1,padding = "same",activation = "relu",use_bias = 1))
CNNmodel.add(layers.Conv2D(1,(1,1),strides = 1))
CNNmodel.add(layers.Flatten())
CNNmodel.load_weights("trainingMhex3rditer8layers26inputs/cp-0010.ckpt")
CNNmodel.save("savedModels/14Inputs")

INFO:tensorflow:Assets written to: savedModels/14Inputs\assets


In [6]:
new_model = tf.keras.models.load_model('savedModels/2Inputs')
np.unravel_index(np.argmax(np.reshape(new_model.predict(positions[5:6]),(13,13)),axis=None),(13,13))

NameError: name 'positions' is not defined

In [ ]:
CNNmodel.summary()

In [ ]:
data = np.load('data/scoredPositionsFull.npz')
actionData = np.load('data/actions.npz')
positions = data['positions']
positions = positions.transpose(0,2,3,1)

actions = actionData['actions']
actions = np.delete(actions,0,1)
actions = np.delete(actions,0,1)

actions = np.delete(actions,14,1)
actions = np.delete(actions,13,1)
actions = np.delete(actions,0,2)
actions = np.delete(actions,0,2)

actions = np.delete(actions,14,2)
actions = np.delete(actions,13,2)
actions = np.reshape(actions,(539998,169))
actions = np.where(actions == 1)
actions = actions[1]
actions

In [ ]:
opt = optimizers.Adam()
CNNmodel.compile(optimizer = opt, loss = tf.keras.losses.SparseCategoricalCrossentropy(),metrics = [metrics.SparseCategoricalAccuracy()])

In [ ]:
checkpoint_path = "trainingSGD/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath = checkpoint_path,verbose = 1, save_weights_only = True, save_freq = 3985)

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

In [ ]:
positions_test = positions[-30000:]
actions_test = actions[-30000:]
positions_train = positions[:-30000]
actions_train = actions[:-30000]

In [ ]:
print(positions[:20][0])
print(positions[:20][0])

In [ ]:
trained = CNNmodel.fit(positions_train,actions_train,batch_size = 128, epochs = 50,validation_data= (positions_test,actions_test),callbacks= [cp_callback,callback,tensorboard_callback])

In [5]:
output_dict[6].history

{'loss': [2.7788455486297607,
  2.8180606365203857,
  2.7739062309265137,
  2.74249267578125,
  2.7072224617004395,
  2.7111639976501465,
  2.6776983737945557,
  2.6646854877471924,
  2.6612935066223145,
  2.642488956451416,
  2.595977544784546,
  2.6104578971862793,
  2.6225745677948,
  2.5721933841705322,
  2.595229148864746,
  2.536811590194702,
  2.5223264694213867,
  2.505760431289673,
  2.5640907287597656,
  2.490449905395508,
  2.5371432304382324,
  2.4701790809631348,
  2.4989943504333496,
  2.435443639755249,
  2.4067840576171875,
  2.4105587005615234,
  2.3761987686157227,
  2.4080262184143066,
  2.4449756145477295,
  2.4158260822296143,
  2.4274518489837646,
  2.4598395824432373],
 'sparse_categorical_accuracy': [0.2682676911354065,
  0.2665562629699707,
  0.2716096043586731,
  0.2749677002429962,
  0.27827343344688416,
  0.2773105800151825,
  0.28132107853889465,
  0.2841953635215759,
  0.2848201394081116,
  0.2884049117565155,
  0.2923239767551422,
  0.29238492250442505,
 

In [ ]:
history = output_dict[2]
plt.plot(history.history['sparse_categorical_accuracy'])
plt.plot(history.history['val_sparse_categorical_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')


In [3]:
plt.savefig('2HiddenLayer128BatchDoubleData2Inputs.png', dpi=400)

<Figure size 432x288 with 0 Axes>

In [1]:
import tensorflow as tf



In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12412811887359791751
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 6972309504
locality {
  bus_id: 1
  links {
  }
}
incarnation: 9364139211471760199
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1070, pci bus id: 0000:01:00.0, compute capability: 6.1"
xla_global_id: 416903419
]


In [1]:
import tensorflow as tf
tf.config.run_functions_eagerly(True)
from tensorflow.keras import layers,models,metrics,optimizers,initializers
import matplotlib.pyplot as plt
import numpy as np
import os
import math

data = np.load('data/mohexPositions.npz')
actionData = np.load('data/mohexActions.npz')
positions = data['positions']
positions = positions.transpose(0,2,3,1)
# positions = np.delete(positions,np.s_[2:18],3)


actions = actionData['actions']
actions = np.delete(actions,0,1)
actions = np.delete(actions,0,1)

actions = np.delete(actions,14,1)
actions = np.delete(actions,13,1)
actions = np.delete(actions,0,2)
actions = np.delete(actions,0,2)

actions = np.delete(actions,14,2)
actions = np.delete(actions,13,2)

actions = np.reshape(actions,(1760832,169))

actions = np.where(actions == 1)

actions = actions[1]



# positions_train =np.concatenate((positions[:-30000], positions_mirror), axis= 0) 
# actions_train =np.concatenate((actions[:-30000], actions_mirror), axis= 0) 
inputs = [(1,1),(2,18),(6,18),(2,6)] 
for k in range (3,4):
    if k > 0: 
        positions = np.delete(positions,np.s_[inputs[k][0]:inputs[k][1]],3)
    positions_test = positions[-200000:] 
    actions_test = actions[-200000:]
    actions_train = actions[500000:-200000]
    positions_train = positions[500000:-200000]
    for i in range(8,9):

        model = models.Sequential()
    #     init_stddev = math.sqrt(2.0 / (18 * 5 * 5))
        if k > 0:
            model.add(layers.Conv2D(128,(5,5),strides = 1, activation ="relu",padding = "valid" ,input_shape=(17,17,18 - (inputs[k][1] - inputs[k][0]))))
        else:
            model.add(layers.Conv2D(128,(5,5),strides = 1, activation ="relu",padding = "valid" ,input_shape=(17,17,18)))
       
        for j in range (0,i):
    #         init_stddev = math.sqrt(2.0 / (128 * 3 * 3))
            model.add(layers.Conv2D(128,(3,3),strides =1,padding = "same",activation = "relu",use_bias = 1))

        model.add(layers.Conv2D(1,(1,1),strides = 1))
        model.add(layers.Flatten())
        model.load_weights("trainingMhex2nditer8layers26inputs/cp-0020.ckpt")
            
        checkpoint_path = "trainingMhex3rditer" + str(i) + "layers" + str(inputs[k][0]) + str(inputs[k][1]) +  "inputs/cp-{epoch:04d}.ckpt"
        checkpoint_dir = os.path.dirname(checkpoint_path)
        cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath = checkpoint_path,verbose = 1, save_weights_only = True, save_freq = 8204 * 5)
        opt = optimizers.Adam(0.00001)
        callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=4)

        model.compile(optimizer = opt, loss = tf.keras.losses.SparseCategoricalCrossentropy(),metrics = [metrics.SparseCategoricalAccuracy()])
        output = model.fit(positions_train,actions_train,batch_size = 128, epochs = 100,validation_data= (positions_test,actions_test),callbacks= [cp_callback,callback])
        np.save("history/historyMhex3rditer" + str(i) + "layers" + str(k), output.history)

        
            
    
#         checkpoint_path = "training" + str(i) + "layers2nditer" + str(inputs[k][0]) + str(inputs[k][1]) +  "inputs/cp-{epoch:04d}.ckpt"
#         checkpoint_dir = os.path.dirname(checkpoint_path)
#         cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath = checkpoint_path,verbose = 1, save_weights_only = True, save_freq = 8204 * 5)
#         opt = optimizers.Adam(0.0001)
#         callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=4)


#         model.compile(optimizer = opt, loss = tf.keras.losses.SparseCategoricalCrossentropy(),metrics = [metrics.SparseCategoricalAccuracy()])
#         output = model.fit(positions_train,actions_train,batch_size = 128, epochs = 40,validation_data= (positions_test,actions_test),callbacks= [cp_callback,callback])
#         np.save("history/history2nd" + str(k), output.history)

C:\Users\Joe\AppData\Local\Programs\Python\Python310\lib\site-packages\tensorflow\python\data\ops\structured_function.py:264: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


Epoch 1/100
8288/8288 [==============================] - 413s 48ms/step - loss: 1.9536 - sparse_categorical_accuracy: 0.4999 - val_loss: 1.8520 - val_sparse_categorical_accuracy: 0.5143
Epoch 2/100
8288/8288 [==============================] - 400s 48ms/step - loss: 1.9227 - sparse_categorical_accuracy: 0.5043 - val_loss: 1.8300 - val_sparse_categorical_accuracy: 0.5143
Epoch 3/100
8288/8288 [==============================] - 400s 48ms/step - loss: 2.1974 - sparse_categorical_accuracy: 0.4664 - val_loss: 2.0259 - val_sparse_categorical_accuracy: 0.4872
Epoch 4/100
8288/8288 [==============================] - 400s 48ms/step - loss: 1.8991 - sparse_categorical_accuracy: 0.5077 - val_loss: 1.8675 - val_sparse_categorical_accuracy: 0.5078
Epoch 5/100
7866/8288 [===========================>..] - ETA: 18s - loss: 1.9073 - sparse_categorical_accuracy: 0.5076
Epoch 5: saving model to trainingMhex3rditer8layers26inputs\cp-0005.ckpt
8288/8288 [==============================] - 401s 48ms/step - lo

In [9]:
output.history


{'loss': [3.670104742050171,
  3.2186014652252197,
  3.1971969604492188,
  3.057480573654175,
  3.0015339851379395,
  2.970580816268921,
  2.939509630203247,
  3.55944561958313,
  4.062826633453369,
  3.9119772911071777,
  4.153787136077881],
 'sparse_categorical_accuracy': [0.1799597293138504,
  0.21755321323871613,
  0.20847412943840027,
  0.22829705476760864,
  0.23758375644683838,
  0.24236854910850525,
  0.24763236939907074,
  0.2021617293357849,
  0.16003775596618652,
  0.15984536707401276,
  0.15454821288585663],
 'val_loss': [4.323082447052002,
  2.963268756866455,
  2.9706785678863525,
  2.930213451385498,
  2.8059346675872803,
  2.7547059059143066,
  3.3326892852783203,
  3.3128318786621094,
  4.795958995819092,
  5.0748515129089355,
  3.510409355163574],
 'val_sparse_categorical_accuracy': [0.13866665959358215,
  0.2365666627883911,
  0.23313333094120026,
  0.2489333301782608,
  0.25966668128967285,
  0.2720000147819519,
  0.18310000002384186,
  0.18596667051315308,
  0.1004

In [1]:


import tensorflow as tf
tf.config.run_functions_eagerly(True)
from tensorflow.keras import layers,models,metrics,optimizers,initializers
import matplotlib.pyplot as plt
import numpy as np
import os
import math
%load_ext tensorboard
data = np.load('data/scoredPositionsFull.npz')
actionData = np.load('data/actions.npz')
positions = data['positions']
positions = positions.transpose(0,2,3,1)
# positions = np.delete(positions,np.s_[2:18],3)
positions_mirror = positions.transpose(0,2,1,3)

actions = actionData['actions']
actions = np.delete(actions,0,1)
actions = np.delete(actions,0,1)

actions = np.delete(actions,14,1)
actions = np.delete(actions,13,1)
actions = np.delete(actions,0,2)
actions = np.delete(actions,0,2)

actions = np.delete(actions,14,2)
actions = np.delete(actions,13,2)
actions_mirror = actions.transpose(0,2,1)
actions = np.reshape(actions,(539998,169))
actions_mirror = np.reshape(actions_mirror, (539998,169))
actions = np.where(actions == 1)
actions_mirror = np.where(actions_mirror == 1)
actions = actions[1]
actions_mirror = actions_mirror[1]

positions_test = positions[-30000:] 
actions_test = actions[-30000:]
positions_train =np.concatenate((positions[:-30000], positions_mirror), axis= 0) 
actions_train =np.concatenate((actions[:-30000], actions_mirror), axis= 0) 
inputs = [(1,1),(2,18),(6,18),(2,6)] 
for k in range (0,1):
    
    positions = data['positions']
    positions = positions.transpose(0,2,3,1)
    if k > 0: 
        positions = np.delete(positions,np.s_[inputs[k][0]:inputs[k][1]],3)
    positions_mirror = positions.transpose(0,2,1,3)
    positions_test = positions[-30000:] 
    positions_train =np.concatenate((positions[:-30000], positions_mirror), axis= 0) 
    for i in range(6,7):

        model = models.Sequential()
    #     init_stddev = math.sqrt(2.0 / (18 * 5 * 5))
        if k > 0:
            model.add(layers.Conv2D(128,(5,5),strides = 1, activation ="relu",padding = "valid" ,input_shape=(17,17,18 - (inputs[k][1] - inputs[k][0]))))
        else:
            model.add(layers.Conv2D(128,(5,5),strides = 1, activation ="relu",padding = "valid" ,input_shape=(17,17,18)))
       
        for j in range (0,i):
    #         init_stddev = math.sqrt(2.0 / (128 * 3 * 3))
            model.add(layers.Conv2D(128,(3,3),strides =1,padding = "same",activation = "relu",use_bias = 1))

        model.add(layers.Conv2D(1,(1,1),strides = 1))
        model.add(layers.Flatten())
        model.load_weights("DataModels/18 Inputs/DoubleDataWeights/training66layers3rdIter/cp-0027.ckpt")
            
        checkpoint_path = "training" + str(i) + "layers6thiter" + str(inputs[k][0]) + str(inputs[k][1]) +  "inputs/cp-{epoch:04d}.ckpt"
        checkpoint_dir = os.path.dirname(checkpoint_path)
        cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath = checkpoint_path,verbose = 1, save_weights_only = True, save_freq = 8204 * 5)
        opt = optimizers.Adam(0.00001)
        callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=4)

        model.compile(optimizer = opt, loss = tf.keras.losses.SparseCategoricalCrossentropy(),metrics = [metrics.SparseCategoricalAccuracy()])
        output = model.fit(positions_train,actions_train,batch_size = 128, epochs = 60,validation_data= (positions_test,actions_test),callbacks= [cp_callback,callback])
        np.save("history/history6thiter" + str(k), output.history)

C:\Users\Joe\AppData\Local\Programs\Python\Python310\lib\site-packages\tensorflow\python\data\ops\structured_function.py:264: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


Epoch 1/60
8204/8204 [==============================] - 295s 36ms/step - loss: 2.4530 - sparse_categorical_accuracy: 0.3164 - val_loss: 2.4333 - val_sparse_categorical_accuracy: 0.3209
Epoch 2/60
8204/8204 [==============================] - 292s 36ms/step - loss: 2.3764 - sparse_categorical_accuracy: 0.3305 - val_loss: 2.3937 - val_sparse_categorical_accuracy: 0.3289
Epoch 3/60
8204/8204 [==============================] - 292s 36ms/step - loss: 2.4352 - sparse_categorical_accuracy: 0.3219 - val_loss: 2.4286 - val_sparse_categorical_accuracy: 0.3209
Epoch 4/60
8204/8204 [==============================] - 295s 36ms/step - loss: 2.3984 - sparse_categorical_accuracy: 0.3299 - val_loss: 2.4270 - val_sparse_categorical_accuracy: 0.3192
Epoch 5/60
8202/8204 [============================>.] - ETA: 0s - loss: 2.4383 - sparse_categorical_accuracy: 0.3210
Epoch 5: saving model to training6layers6thiter11inputs\cp-0005.ckpt
8204/8204 [==============================] - 296s 36ms/step - loss: 2.4383

In [15]:
model = models.Sequential()

In [17]:
model.load_weights("Hex data\MoHex-CNN model\slmodel.ckpt-135000")

Instructions for updating:
Restoring a name-based tf.train.Saver checkpoint using the object-based restore API. This mode uses global names to match variables, and so is somewhat fragile. It also adds new restore ops to the graph each time it is called when graph building. Prefer re-encoding training checkpoints in the object-based format: run save() on the object-based saver (the same one this message is coming from) and use that checkpoint in the future.


In [20]:
model.build()

ValueError: You must provide an `input_shape` argument.

In [9]:
data1 = np.load('data/scoredPositionsMohex21.npz')
data2 = np.load('data/scoredPositionsMohex22.npz')
data3 = np.load('data/scoredPositionsMohex23.npz')
data4 = np.load('data/scoredPositionsMohex24.npz')
data5 = np.load('data/scoredPositionsMohex25.npz')
data6 = np.load('data/scoredPositionsMohex26.npz')
data7 = np.load('data/scoredPositionsMohex27.npz')
data8 = np.load('data/scoredPositionsMohex28.npz')

positions = np.concatenate((data1['positions'],data2['positions'],data3['positions'],data4['positions'],data5['positions'],data6['positions'],data7['positions'],data8['positions']))

In [10]:
np.savez( "data/mohexPositions.npz", positions = positions)

In [3]:
data1 = np.load('data/positions0.npz')
data2 = np.load('data/positions1000.npz')
data3= np.load('data/positions2000.npz')
data4 = np.load('data/positions3000.npz')
data5 = np.load('data/positions4000.npz')
data6 = np.load('data/positions5000.npz')
data7 = np.load('data/positions6000.npz')
data8 = np.load('data/positions7000.npz')
data9 = np.load('data/positions8000.npz')
data10 = np.load('data/positions9000.npz')
data11 = np.load('data/positions10000.npz')
data12 = np.load('data/positions11000.npz')
data13 = np.load('data/positions12000.npz')
data14 = np.load('data/positions13000.npz')
data15 = np.load('data/positions14000.npz')
data16 = np.load('data/positions15000.npz')
data17 = np.load('data/positions16000.npz')
data18 = np.load('data/positions17000.npz')
data19 = np.load('data/positions18000.npz')
data20 = np.load('data/positions19000.npz')
data21 = np.load('data/positions20000.npz')
data22 = np.load('data/positions21000.npz')
data23 = np.load('data/positions22000.npz')
data24 = np.load('data/positionsfinal.npz')


actions = np.concatenate((data1['positions'],data2['positions'],data3['positions'],data4['positions'],data5['positions'],data6['positions'],data7['positions'],data8['positions'],data9['positions'],data10['positions'],data11['positions'],data12['positions'],data13['positions'],data14['positions'],data15['positions'],data16['positions'],data17['positions'],data18['positions'],data19['positions'],data20['positions'],data21['positions'],data22['positions'],data23['positions'],data24['positions']))
np.savez("data/mohexPositions.npz",positions = actions)

In [15]:
actionData = np.load('data/mohexActions.npz')
actions = actionData['actions']
np.shape(actions)

(1760832, 17, 17)